In [5]:
import json
import librosa
import soundfile as sf
import numpy as np
import os
from glob import glob

In [16]:
for audio_fname in glob("./*/*.wav"):
    audio, sr = librosa.load(audio_fname, sr = None)
    if "train" in audio_fname:
        dur = 100
    else:
        dur = 50
        
    audio = audio[ :int(sr * dur) ]
    sf.write(audio_fname, audio, samplerate=sr )
    
    if "channel_1" in audio_fname:
        label_fname = audio_fname[:-4] + ".json"
        label = json.load(open(label_fname))
        onset = np.array(label["onset"])
        offset = np.array(label["offset"])
        valid_indices = onset < len(audio)/sr
        onset = onset[valid_indices]
        offset = offset[valid_indices]
        offset[offset>len(audio)/sr] = len(audio)/sr
        label["onset"] = onset.tolist()
        label["offset"] = offset.tolist()
        label["cluster"] = [ "vocal" ] * len( onset )

        json.dump( label, open(label_fname, "w") )